In [90]:
import re

import keras
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, LSTM
from collections import defaultdict

In [91]:
text = open('text.txt').read().lower()

text = re.sub(r'[ё]', 'е', text)
text = re.sub(r'[\,\:\;\–\-\«\»\!\)\(\?\"\'\…ivx]', '', text)
# text = re.sub(r'[^а-я\s.]', '.', text)
text = re.sub(r'\d+', '', text)
text = re.sub(r'\s+', ' ', text)
text = re.sub(r'\s+\.', '.', text)
print(text)

принимая во внимание показатели успешности реализация намеченных плановых заданий играет определяющее значение для соответствующих условий активизации. в своем стремлении улучшить пользовательский опыт мы упускаем что стремящиеся вытеснить традиционное производство нанотехнологии объективно рассмотрены соответствующими инстанциями. сложно сказать почему реплицированные с зарубежных источников современные исследования в равной степени предоставлены сами себе. идейные соображения высшего порядка а также убежденность некоторых оппонентов позволяет оценить значение первоочередных требований. имеется спорная точка зрения гласящая примерно следующее тщательные исследования конкурентов инициированные исключительно синтетически описаны максимально подробно. являясь всего лишь частью общей картины представители современных социальных резервов и по сей день остаются уделом либералов которые жаждут быть призваны к ответу. мы вынуждены отталкиваться от того что постоянный количественный рост и сфе

In [92]:
text_chars = sorted(list(set(text)))
chars = {}
indexes = {}

for ind, char in enumerate(text_chars):
    chars[char] = ind
    indexes[ind] = char

print(chars)
print(indexes)

{' ': 0, '.': 1, 'а': 2, 'б': 3, 'в': 4, 'г': 5, 'д': 6, 'е': 7, 'ж': 8, 'з': 9, 'и': 10, 'й': 11, 'к': 12, 'л': 13, 'м': 14, 'н': 15, 'о': 16, 'п': 17, 'р': 18, 'с': 19, 'т': 20, 'у': 21, 'ф': 22, 'х': 23, 'ц': 24, 'ч': 25, 'ш': 26, 'щ': 27, 'ъ': 28, 'ы': 29, 'ь': 30, 'э': 31, 'ю': 32, 'я': 33}
{0: ' ', 1: '.', 2: 'а', 3: 'б', 4: 'в', 5: 'г', 6: 'д', 7: 'е', 8: 'ж', 9: 'з', 10: 'и', 11: 'й', 12: 'к', 13: 'л', 14: 'м', 15: 'н', 16: 'о', 17: 'п', 18: 'р', 19: 'с', 20: 'т', 21: 'у', 22: 'ф', 23: 'х', 24: 'ц', 25: 'ч', 26: 'ш', 27: 'щ', 28: 'ъ', 29: 'ы', 30: 'ь', 31: 'э', 32: 'ю', 33: 'я'}


In [93]:
previous_length = 10
dataX = []
dataY = []
text_sentences = text.split('.')
for sentence in text_sentences:
    for i in range(len(sentence) - previous_length):
        dataX.append(sentence[i:i + previous_length])
        dataY.append(sentence[previous_length + i])

print(dataX[:40])
print(dataY[:40])

['принимая в', 'ринимая во', 'инимая во ', 'нимая во в', 'имая во вн', 'мая во вни', 'ая во вним', 'я во внима', ' во вниман', 'во внимани', 'о внимание', ' внимание ', 'внимание п', 'нимание по', 'имание пок', 'мание пока', 'ание показ', 'ние показа', 'ие показат', 'е показате', ' показател', 'показатели', 'оказатели ', 'казатели у', 'азатели ус', 'затели усп', 'атели успе', 'тели успеш', 'ели успешн', 'ли успешно', 'и успешнос', ' успешност', 'успешности', 'спешности ', 'пешности р', 'ешности ре', 'шности реа', 'ности реал', 'ости реали', 'сти реализ']
['о', ' ', 'в', 'н', 'и', 'м', 'а', 'н', 'и', 'е', ' ', 'п', 'о', 'к', 'а', 'з', 'а', 'т', 'е', 'л', 'и', ' ', 'у', 'с', 'п', 'е', 'ш', 'н', 'о', 'с', 'т', 'и', ' ', 'р', 'е', 'а', 'л', 'и', 'з', 'а']


In [94]:
count_sentences = len(dataX)
X = np.zeros((count_sentences, previous_length, len(chars)))
Y = np.zeros((count_sentences, len(chars)))
for i, sentence in enumerate(dataX):
    for j, char in enumerate(sentence):
        X[i, j, chars[char]] = 1
    Y[i, chars[dataY[i]]] = 1

print(np.argmax(X[0]), indexes[np.argmax(X[0])])
print(f'one hot: {X[0][0]} len: {len(X[0][0])}')

17 п
one hot: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] len: 34


In [95]:
print(X.shape)
print(Y.shape)

(49831, 10, 34)
(49831, 34)


In [96]:
filepath="weights/improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model = keras.Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(Y.shape[1], activation='softmax'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 128)               83456     
                                                                 
 dense_8 (Dense)             (None, 34)                4386      
                                                                 
Total params: 87,842
Trainable params: 87,842
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy')
history = model.fit(X, Y, epochs=30, callbacks=callbacks_list)

Epoch 1/30
1554/1558 [============================>.] - ETA: 0s - loss: 2.5298
Epoch 1: loss improved from inf to 2.52898, saving model to weights\improvement-01-2.5290-bigger.hdf5
1558/1558 [==============================] - 17s 9ms/step - loss: 2.5290
Epoch 2/30
1555/1558 [============================>.] - ETA: 0s - loss: 1.9146
Epoch 2: loss improved from 2.52898 to 1.91401, saving model to weights\improvement-02-1.9140-bigger.hdf5
1558/1558 [==============================] - 15s 10ms/step - loss: 1.9140
Epoch 3/30
1554/1558 [============================>.] - ETA: 0s - loss: 1.4283
Epoch 3: loss improved from 1.91401 to 1.42790, saving model to weights\improvement-03-1.4279-bigger.hdf5
1558/1558 [==============================] - 14s 9ms/step - loss: 1.4279
Epoch 4/30
1557/1558 [============================>.] - ETA: 0s - loss: 0.9832
Epoch 4: loss improved from 1.42790 to 0.98329, saving model to weights\improvement-04-0.9833-bigger.hdf5
1558/1558 [==============================] -

In [83]:
for _ in range(5):
    random_index = np.random.randint(count_sentences - 1)
    current_pattern = dataX[random_index]
    print(f'Pattern is {current_pattern}')
    ans = current_pattern
    for _ in range(30):
        x = np.zeros((1, previous_length, len(text_chars)))
        for i, char in enumerate(current_pattern):
            x[0, i, chars[char]] = 1
        predict = model.predict(x)
        next = indexes[np.argmax(predict)]
        current_pattern = current_pattern[1:] + next
        ans += next

    print(f'Predict is {ans}')
    print('-----------------')

Pattern is вителям эм
1/1 [==============================] - 0s 16ms/step
Predict is вителям эмпоните коночений способствует 
-----------------
Pattern is и способст
1/1 [==============================] - 0s 23ms/step
Predict is и способствует повышению качества форм в
-----------------
Pattern is мократичес
1/1 [==============================] - 0s 10ms/step
Predict is мократической анафемемительные выводы ра
-----------------
Pattern is ссенцию по
1/1 [==============================] - 0s 24ms/step
Predict is ссенцию победы маркетинга над разумом и 
-----------------
Pattern is  все меньш
1/1 [==============================] - 0s 23ms/step
Predict is  все меньше твланкт пробые интелекивой о
-----------------


In [88]:
continues = defaultdict(lambda: defaultdict(lambda: 0))
for sentence, continuee in zip(dataX, dataY):
    continues[sentence][continuee] += 1

for con in dataX:
    l = len(continues[con])
    s = sum([v for v in continues[con].values()])
    for v in continues[con]:
        continues[con][v] /= s
# print(continues)

In [89]:
for _ in range(5):
    random_index = np.random.randint(count_sentences - 1)
    current_pattern = dataX[random_index]

    print(f'Pattern is {current_pattern}')
    ans = current_pattern
    for _ in range(30):
        nexts = continues[current_pattern]
        p = [v for v in nexts.values()]
        #next = list(nexts.items())[np.argmax(p)]
        if len(p) == 0:
            break
        next = list(nexts.items())[np.random.choice(len(p), p=np.array(p))]
        current_pattern = current_pattern[1:] + next[0]
        ans += next[0]
    print(f'Predict is {ans}')
    print('-----------------')

Pattern is дены вводи
Predict is дены вводить в содержательную расшифровк
-----------------
Pattern is ункциональ
Predict is ункционально разнесены на независимые го
-----------------
Pattern is аждут быть
Predict is аждут быть подвергнуты целой серии незав
-----------------
Pattern is налитики в
Predict is налитики выводы подвергнуты целой серии 
-----------------
Pattern is утренней п
Predict is утренней политики неоднозначны и будут о
-----------------
